# How to use MCDM methods
In this example, we will see how MCDM methods available in DESDEO can be
utilized to solve a multiobjective optimization problem. MCDM methods are
implemented in a functional way, which means their individual components must be
first combined. This might feel unnecessary at first, but in practice, it means
we are able to combine components of multiple methods to create new methods that
can best suit our needs and the needs of decision makers.

Since scalarization sits often in a central role in DESDEO, we will first see
examples of how it is done in DESDEO. Then, we will see examples of the
Reference Point Method and Synchronous NIMBUS in action. Throughout the this
example, we will be solving the [river pollution
problem](../../api/desdeo_problem/#desdeo.problem.testproblems.river_pollution_problem),
consisting of two continuous variables and four objective functions ($f_1, f_2,
f_3, f_4$ to be maximized). We choose this problem because
of its relative simplicity.

## Scalarization in DESDEO
Scalarization is the transformation of a multiobjective optimization problem
into a single-objective optimization one. In DESDEO, this happens by adding a
`ScalarizationFunction` to an instance of the `Problem` class. However, because
many scalarization functions require information on either the ideal or nadir
point, or both, of the problem being solved, we first need to compute these
before proceeding with scalarization.

### Computing the ideal point and (approximating) the nadir point
To compute the ideal point, and approximate the nadir point, of the rive pollution problem, 
we can utilize the payoff table method
([`payoff_table_method`](../../api/desdeo_tools/#desdeo.tools.utils.payoff_table_method)).
We begin by creating an instance of the river pollution problem and
passing it to the payoff table method:

In [34]:
from desdeo.problem.testproblems import river_pollution_problem
from desdeo.tools import PyomoIpoptSolver, payoff_table_method

problem = river_pollution_problem(five_objective_variant=False)  # we use the four objective variant

ideal, nadir = payoff_table_method(problem, solver=PyomoIpoptSolver)

While the computed `ideal` point will have its components at the true ideal
points, as the nadir point will generally be a very rough approximation of the
true nadir point due to the nature of the payoff table method. We also notice
that we passed the
[`PyomoIpoptSolver`](../../api/desdeo_tools/#desdeo.tools.pyomo_solver_interfaces.PyomoIpoptSolver)
solver to the payoff table method. This solver interfaces to the
[Ipopt](https://coin-or.github.io/Ipopt/) from the COIN-OR project though a
Pyomo model (https://www.pyomo.org/), hence the name. Ipopt is suitable for
non-linear optimization.

In practice, the nadir point can be either inquired from a decision maker (domain
expert), or it can be approximated by solving
the problem utilizing an evolutionary method, such as NSGA3, and then reading
the nadir point values of the approximated Pareto front. However, while inaccurate,
the payoff table method works out-of-the-box with almost any problem, and it 
gives at least a first workable approximation fo the nadir point. The ideal point
computed by the payoff table method is often accurate enough, however.

<div class="admonition note">
<p class="admonition-title">Note</p>
<p>The payoff table method is a very rough
way to approximate the nadir point of a problem. It is better to use an
evolutionary method instead to approximate the problem's Pareto front, and then
read an approximated value for the nadir point from the front.
</p>
</div>

Regardless the source of the information on the nadir (and ideal) point values,
we can update the problem with this information. For the purpose of this example,
the nadir point approximated by the payoff table method is accurate enough.

In [35]:
problem = problem.update_ideal_and_nadir(ideal, nadir)

It is important to notice we the utilized the method `update_ideal_and_nadir`
and then used its output to redefine the variable `problem`. In DESDEO, instances
of the `Problem` class are, in principle, __immutable__. This means that anytime
we wish to change an already instantiate `Problem` object, we essentially have
to create a new one. `Problem`s are chosen to be immutable in DESDEO to avoid
involuntarily changing the original problem, which can easily happen when
solving the problem with multiple methods.

<div class="admonition note">
<p class="admonition-title">Note</p>
<p>Instances of the <tt>Problem</tt> class in DESDEO are immutable. When making
changes to an existing `Problem` objects, methods and functions will return a
new instance of the original <tt>Problem</tt> with the applied changes.</p>
</div>

### Scalarizing a problem
Once we have the ideal and nadir point values available, we can scalarize our
problem. We will use the achievement scalarizing function, or ASF. To utilize
the ASF, we will need a reference point. Let us use the values $\left[5.55,
3.20, 3.91, -4.85\right]$ for the reference point, as an example. Because the
DTLZ2 problem is differentiable, we can use the differentiable variant of the
ASF ([`desdeo.tools.add_asf_diff`](../../api/desdeo_tools/#desdeo.tools.scalarization.add_asf_diff)):

In [36]:
from desdeo.tools import add_asf_diff

reference_point = {
    "f_1": 5.55,
    "f_2": 3.20,
    "f_3": 3.91,
    "f_4": -4.85,
}

problem_w_asf, target = add_asf_diff(problem, symbol="asf", reference_point=reference_point)

When adding the ASF, we had to supply also a `symbol` in addition to the
`reference_point`. As discussed in the previous example, the `symbol` is
utilized to identify various components of a `Problem`. In this case, the
`symbol="asf"` is used to refer to the ASF, which was added to the problem.

We also mentioned that we can use the differentiable variant of the ASF, because
our problem is differentiable. This is important because we want to keep
the differentiability of our problem even after scalarization. This allows us to
use, e.g., gradient-based optimizers when solving the scalarized problem. In
turn, this allows us to get accurate and optimal solutions. If our problem was
not differentiable, then we could have used the non-differentiable variant of
ASF defined in `desdeo.tools.add_asf_nondiff`. This would be practical if we
knew we had to solve our problem using heuristics-based methods, which are
impartial to the differentiability of our problem. In fact, it could be even
detrimental to utilize the differentiable variant when solving the scalarized
problem with an evolutionary method, since the differentiable variant introduced
many constraints to the problem, which evolutionary methods are not very adept
to handle. The lesson here is, that we should be knowledgeable enough about our
problem to manipulate it in the best possible way.

### Solving a scalarized problem

To solve the scalarized problem, we can once again utilize the `PyomoIpoptSolver`
interface. However, this time we will have to set it up ourselves instead of
just passing it as an argument to the payoff table method:

In [37]:
from pprint import pprint

In [38]:
solver = PyomoIpoptSolver(problem_w_asf)

result = solver.solve(target)

Notice how we passed the `target` to the `solve` method. This tells the solver, which
objective function should be optimized. The solver interfaces in DESDEO
return their optimization results in a Pydantic object. We can inspect its content
by printing it as a dictionary:

In [ ]:
pprint(result.__dict__, width=120)

{'constraint_values': {'f_1_con': -0.3057121235731717,
                       'f_2_con': 5.254153390144012e-09,
                       'f_3_con': -9.922039873044852e-08,
                       'f_4_con': 4.461189334614701e-09},
 'extra_func_values': None,
 'message': "Pyomo solver status is: 'ok', with termination condition: 'optimal'.",
 'optimal_objectives': {'f_1': 6.209215255403704,
                        'f_2': 3.264552137173013,
                        'f_3': 4.693571830352962,
                        'f_4': -3.7905236281284678},
 'optimal_variables': {'_alpha': -0.1091493197477763, 'x_1': 0.9423855750677106, 'x_2': 0.9422934861100473},
 'scalarization_values': {'asf': -0.10915901059978278},
 'success': True}


Out of the results, we are likely most interested in the fields
`optimal_objectives` and `optimal_variables`, which represent the optimal
objective function and variable values for the found optimal solution of the
scalarized problem. The `success` field will indicate whether the optimization
was successful, and the `message` field will yield solver-specific information
on the success or failure of the optimization process. Lastly, checking
the `constraint_values` field can be valuable as well, since as we recall from the
previous example, constraints in DESDEO are defined such that a negative value
means the constraint holds, and a positive that it is breached. Here we see
some values being positive, yet extremely close to zero. In practice, these
values can be safely assumed to be zero due to the nature and precision
of floating-point arithmetics.

## Utilizing the Reference Point Method

The [Reference Point
method](../../api/desdeo_mcdm/#desdeo.mcdm.reference_point_method) is a
relatively simple interactive multiobjective optimization method, which utilizes
scalarization, and a reference point as preference information. The method uses
the reference point to create $k+1$, with $k$ being the number of objective
functions, sub-problems, which use a perturbed version of the original reference
point (one of the sub-problems uses the original unperturbed reference point as
well) to scalarize the problem being solved using the achievement scalarizing
function. The resulting solutions from the sub-problems will be more spread out
the farther the original reference point is from the true Pareto front of the
problem. This can allow a decision maker to get a more general picture of what
kind of solutions are available, when far from the front; and fine-tune their
solution once the decision maker begins to close on the Pareto front with the
given reference point. The method is interactive, which means that a decision
maker is expected to provide reference points iteratively in multiple
during multiple iterations of the method.

As the Reference Point method is quite simple, we will need only one function to
operate it.  We begin by importing it, we choose to use the same
reference point we utilized previously as the initial preference information, and then we
iterate the method:

In [45]:
from desdeo.mcdm import rpm_solve_solutions

results = rpm_solve_solutions(problem, reference_point, solver=PyomoIpoptSolver)

print(f"Number of solutions {len(results)}")
print(f"Original reference point: {reference_point}")
for i, result in enumerate(results):
    print(f"Solution {i + 1}: F={result.optimal_objectives}")

Number of solutions 5
Original reference point: {'f_1': 5.55, 'f_2': 3.2, 'f_3': 3.91, 'f_4': -4.85}
Solution 1: F={'f_1': 6.209215255403704, 'f_2': 3.264552137173013, 'f_3': 4.693571830352962, 'f_4': -3.7905236281284678}
Solution 2: F={'f_1': 6.333581100449999, 'f_2': 3.2320615106376724, 'f_3': 0.786903967588799, 'f_4': -3.086415992189174}
Solution 3: F={'f_1': 6.340000004252907, 'f_2': 3.444871833350942, 'f_3': 0.32111078266553417, 'f_4': -9.706668949083838}
Solution 4: F={'f_1': 6.035263422876624, 'f_2': 3.2609445613715065, 'f_3': 6.12463885963372, 'f_4': -3.8497337702333416}
Solution 5: F={'f_1': 6.240119482700062, 'f_2': 3.2220277112967315, 'f_3': 4.177385479588673, 'f_4': -3.0136070634586956}


As seen from the output, we got 5 solutions, which is expected with 4 objective
functions. Notice also how we passed the instance of the problem without an
added scalarization function (`problem`). The addition of the scalarization
function happened in the function implementing the Reference Point method.

Suppose a decision maker now provides a new reference point: $[6.10, 3.50, 7.1, -5.67]$.
We use it to iterate the method again:

In [47]:
new_reference_point = {"f_1": 6.10, "f_2": 3.50, "f_3": 7.1, "f_4": -5.67}

results = rpm_solve_solutions(problem, new_reference_point, solver=PyomoIpoptSolver)

print(f"Number of solutions {len(results)}")
print(f"Original reference point: {new_reference_point}")
for i, result in enumerate(results):
    print(f"Solution {i + 1}: F={result.optimal_objectives}")

Number of solutions 5
Original reference point: {'f_1': 6.1, 'f_2': 3.5, 'f_3': 7.1, 'f_4': -5.67}
Solution 1: F={'f_1': 6.101764838919299, 'f_2': 3.3889712959545624, 'f_3': 5.752269857223239, 'f_4': -7.492283292660489}
Solution 2: F={'f_1': 6.340000018718053, 'f_2': 3.2332400604525633, 'f_3': 0.3211096655441512, 'f_4': -3.0982539086713006}
Solution 3: F={'f_1': 6.340000003499234, 'f_2': 3.44487183329319, 'f_3': 0.3211108408706522, 'f_4': -9.706668948836672}
Solution 4: F={'f_1': 5.5969121623973255, 'f_2': 3.3147371429058308, 'f_3': 7.096351295672935, 'f_4': -5.378453025905292}
Solution 5: F={'f_1': 6.162954687421054, 'f_2': 3.3476331401795956, 'f_3': 5.250483709221593, 'f_4': -5.901520917863092}


We can now notice how the solutions changed with the new reference point. This
process of providing a reference point and inspecting solutions can continue for
multiple iterations until a satisfactory solution is found, or the decision
maker gets tired, for instance. 


## Utilizing Synchronous NIMBUS

## Conclusions
In practice, we would use at least visualizations to better convey information
about the solutions to a decision maker.  Or even better, a dedicated interface.
However, the purpose of this example is to showcase how the algorithmic aspect
of interactive methods can be utilized. Users are encouraged to utilize the user
interfaces and visualizations provided in the Web-GUI part of DESDEO, or they
own.